# Classificação de Dados com Python 3 - Parte 3

Autor: Rodolfo Bolconte Donato - https://github.com/rodolfobolconte

Data: 13 de Fevereiro de 2020

## Objetivos

- Carregar Conjunto de Dados com informações de Diagnósticos de Câncer de Mama;
- Reamostragem do Conjunto de Dados utilizando o método _K-Fold_;
- Carregar Algoritmos de Aprendizado de Máquina Supervisionado para a Classificão dos Dados (entre Benígno e Malígno);
- Utilizar as melhores combinações de parâmetros dos Algoritmos encontradas;
- Utilizar Métricas Estatísticas voltadas para a mensuração de Modelos de Classificação de Dados.

## 0- Carregando Código

Cópia de trechos do código já utilizado na [Parte 2](https://github.com/rodolfobolconte/minicurso-aprendizado-supervisionado/blob/master/Parte%202.ipynb) para carregar os Algoritmos de Aprendizado de Máquina com as melhores combinações de parâmetros encontradas.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier


#Modelos Multinomial Naive Bayes para serem utilizandos com Reamostragem Holdout e Bootstrap
alpha = 0.5
fit_prior = False
class_prior = [0.25, 0.5]
modelMN = MultinomialNB(alpha=alpha,
                          fit_prior=fit_prior,
                          class_prior=class_prior)

#Modelos Decision Tree para serem utilizandos com Reamostragem Holdout e Bootstrap
criterion = 'entropy' #'gini' ou 'entropy'
splitter = 'random' #'best' ou 'random'
max_depth = 4
min_samples_split = 4
min_samples_leaf = 2
max_features = 'log2' #'auto' == 'sqrt' ou 'log2' ou None
max_leaf_nodes = 8
modelDT = DecisionTreeClassifier(criterion=criterion,
                                   splitter=splitter,
                                   max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   min_samples_leaf=min_samples_leaf,
                                   max_features=max_features,
                                   max_leaf_nodes=max_leaf_nodes)

#Modelos Random Forest para serem utilizandos com Reamostragem Holdout e Bootstrap
n_estimators = 50
criterion = 'entropy' #'gini' ou 'entropy'
max_depth = 4
min_samples_split = 4
min_samples_leaf = 2
max_features = 'log2' #'auto' == 'sqrt' ou 'log2' ou None
max_leaf_nodes = 8
bootstrap = False
max_samples = 0.5 #intervalo entre 0 e 1
modelRF = RandomForestClassifier(n_estimators=n_estimators,
                                   criterion=criterion,
                                   max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   min_samples_leaf=min_samples_leaf,
                                   max_features=max_features,
                                   max_leaf_nodes=max_leaf_nodes,
                                   bootstrap=bootstrap,
                                   max_samples=max_samples)

#Modelos Adaptive Boosting para serem utilizandos com Reamostragem Holdout e Bootstrap
base_estimator = None
n_estimators = 100
learning_rate = 2
algorithm = 'SAMME' # 'SAMME.R' ou 'SAMME'
modelAB = AdaBoostClassifier(base_estimator=base_estimator,
                               n_estimators=n_estimators,
                               learning_rate=learning_rate,
                               algorithm=algorithm)

## 1- Conjunto de Dados

Carregando o Conjunto de Dados e atribuindo as amostras e seus rótulos para duas variáveis a serem utilizadas na Reamostragem.

In [ ]:
from sklearn.datasets import load_breast_cancer

#Carregando o Conjunto de Dados
dataset = load_breast_cancer()

#Colocando os dados em X e os rótulos em Y
X = dataset.data
Y = dataset.target

## 2- Reamostragem e Execução

Reamostragem do Conjunto de Dados utilizando o método _K-Fold_ onde _k_ subconjuntos são gerados _k_ vezes, em que _k_-1 subconjuntos são utilizados para Treino dos Algoritmos e o subconjunto restante é utilizado para Teste a cada execução _k_.

Ao mesmo tempo em que a Reamostragem é feita ao passo de _k_, os algoritmos são Treinados e Testados, visto que guardar os dados a cada iteração _k_ para depois os algoritmos trabalharem com os subconjuntos não é um processo ágil.

Além da execução dos Algoritmos, os seus resultados métricos serão armazenados em suas respectivas variáveis, pelo mesmo motivo da execução dos algoritmos ao passo de cada divisão dos subconjuntos do _K-Fold_.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score

#Quantidade k de subconjuntos e execuções
nK = 5
#Cria o objeto responsável pela divisão do Conjunto
KF = KFold(n_splits=nK)

#Criando e Iniciando as variáveis para armazenar os resultados, que serão incrementados
metricsMN = dict() ; metricsDT = dict() ; metricsRF = dict() ; metricsAB = dict()
metricsMN['confusion'] = 0 ; metricsMN['accuracy'] = 0 ; metricsMN['precision'] = 0 ; metricsMN['recall'] = 0
metricsDT['confusion'] = 0 ; metricsDT['accuracy'] = 0 ; metricsDT['precision'] = 0 ; metricsDT['recall'] = 0
metricsRF['confusion'] = 0 ; metricsRF['accuracy'] = 0 ; metricsRF['precision'] = 0 ; metricsRF['recall'] = 0
metricsAB['confusion'] = 0 ; metricsAB['accuracy'] = 0 ; metricsAB['precision'] = 0 ; metricsAB['recall'] = 0

#No escopo do for a função split retorna os indices da divisão dos subconjuntos para as variáveis de treino e teste
for trainIndex, testIndex in KF.split(Y):
    #Atribui as amostras e seus rótulos para treino e teste no passo k 
    trainX, testX = X[trainIndex], X[testIndex]
    trainY, testY = Y[trainIndex], Y[testIndex]

    #Treino dos algoritmos utilizando as amostras de treino
    modelMN.fit(trainX, trainY)
    modelDT.fit(trainX, trainY)
    modelRF.fit(trainX, trainY)
    modelAB.fit(trainX, trainY)
    
    #Previsão dos algoritmos utilizando os dados de teste
    previsionMN = modelMN.predict(testX)
    previsionDT = modelDT.predict(testX)
    previsionRF = modelRF.predict(testX)
    previsionAB = modelAB.predict(testX)

    #Calcula os resultados para o Multinomial Naive Bayes
    metricsMN['accuracy'] += accuracy_score(testY, previsionMN)
    metricsMN['precision'] += precision_score(testY, previsionMN)
    metricsMN['recall'] += recall_score(testY, previsionMN)

    #Calcula os resultados para o Decision Tree
    metricsDT['accuracy'] += accuracy_score(testY, previsionDT)
    metricsDT['precision'] += precision_score(testY, previsionDT)
    metricsDT['recall'] += recall_score(testY, previsionDT)

    #Calcula os resultados para o Random Forest
    metricsRF['accuracy'] += accuracy_score(testY, previsionRF)
    metricsRF['precision'] += precision_score(testY, previsionRF)
    metricsRF['recall'] += recall_score(testY, previsionRF)

    #Calcula os resultados para o Adaptive Boosting
    metricsAB['accuracy'] += accuracy_score(testY, previsionAB)
    metricsAB['precision'] += precision_score(testY, previsionAB)
    metricsAB['recall'] += recall_score(testY, previsionAB)


#Tirando a média dos Resultados Métricos
metricsMN['accuracy'] /= nK
metricsMN['precision'] /= nK
metricsMN['recall'] /= nK

metricsDT['accuracy'] /= nK
metricsDT['precision'] /= nK
metricsDT['recall'] /= nK

metricsRF['accuracy'] /= nK
metricsRF['precision'] /= nK
metricsRF['recall'] /= nK

metricsAB['accuracy'] /= nK
metricsAB['precision'] /= nK
metricsAB['recall'] /= nK

## 3- Exportando Resultados

Exportar todos os resultados para arquivo .CSV de melhor interpretação e apresentação de valores utilizando um editor de Planilhas.

In [ ]:
csv = open('Parte 3 - Resultados.csv', 'w')

csv.write('Algoritmos;Métricas Bootstrap')
csv.write('\n;Acurácia;Precisão;Sensibilidade')

csv.write("\nMultinomial NB;%.4f;%.4f;%.4f" %(metricsMN['accuracy'], metricsMN['precision'], metricsMN['recall']))

csv.write("\nDecision Tree;%.4f;%.4f;%.4f" %(metricsDT['accuracy'], metricsDT['precision'], metricsDT['recall']))

csv.write("\nRandom Forest;%.4f;%.4f;%.4f" %(metricsRF['accuracy'], metricsRF['precision'], metricsRF['recall']))

csv.write("\nAdaptive Boosting;%.4f;%.4f;%.4f" %(metricsAB['accuracy'], metricsAB['precision'], metricsAB['recall']))